### Importing packages

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.stats import pearsonr,spearmanr
import scipy
import seaborn as sns
import sklearn
from statsmodels.graphics.tsaplots import plot_acf
from statsmodels.tsa.stattools import ccf
from statsmodels.graphics.tsaplots import plot_pacf
import matplotlib.pyplot as plt
from sklearn.metrics import r2_score 
from sklearn import linear_model
from scipy import stats
from sklearn.preprocessing import LabelEncoder
import statistics
import os
from numpy import percentile

## Reading Hospital data

In [ ]:
dataset_dir = './data'

In [ ]:
df = pd.read_csv(os.path.join(dataset_dir, 'hosp_data.csv'))

In [ ]:
df

In [ ]:
#df = pd.read_csv("H:/asthma/data/hosp_data.csv")


In [ ]:
df.columns

In [ ]:
df['Admit Date'] = pd.to_datetime(df['Admit Date']) 

In [ ]:
df = pd.DataFrame(df.loc[(df['Admit Date'] >= '2018-01-01') & (df['Admit Date'] <= '2021-01-01')])

In [ ]:
df = df.sort_values(by='Admit Date').reset_index(drop = True)

In [ ]:
df

In [ ]:
#df = df[df['DHB Desc'] == 'Auckland']

In [ ]:
df_sub = df[df['DHB Desc'] == 'Auckland']

In [ ]:
df_sub

In [ ]:
df_sub = pd.DataFrame(df_sub.groupby(['Admit Date']).size().reset_index(name = "Daily counts")).sort_values(by='Admit Date')

In [ ]:
df_sub

In [ ]:
df_sub_y1 = pd.DataFrame(df_sub.loc[(df_sub['Admit Date'] >= '2018-01-01') & (df_sub['Admit Date'] < '2019-01-01')]).sort_values(by = "Admit Date").reset_index(drop=True)
df_sub_y2 = pd.DataFrame(df_sub.loc[(df_sub['Admit Date'] >= '2019-01-01') & (df_sub['Admit Date'] < '2020-01-01')]).sort_values(by = "Admit Date").reset_index(drop=True)
df_sub_y3 = pd.DataFrame(df_sub.loc[(df_sub['Admit Date'] >= '2020-01-01') & (df_sub['Admit Date'] < '2021-01-01')]).sort_values(by = "Admit Date").reset_index(drop=True)


In [ ]:
df_sub_y1.describe()

In [ ]:
#df_sub_y3.to_csv('H:/asthma/data/count_chk.csv', index = False)

In [ ]:
df

In [ ]:
#df = df[df["Admit CBU Desc"].str.contains("Gen Medicine|CFU|CED|Critical Care|Emergency|Respiratory")]

In [ ]:
#df.to_csv('H:/asthma/data/chking.csv', index = False)

In [ ]:
df = pd.DataFrame(df.groupby(['Admit Date']).size().reset_index(name = "Daily counts")).sort_values(by='Admit Date')

In [ ]:
#df.to_csv('H:/asthma/data/hosp_chk.csv', index = False)

In [ ]:
idx = pd.date_range(start = '2018-01-01', end = '2021-01-01' ).difference(df['Admit Date'])

In [ ]:
idx

In [ ]:
df1 = pd.DataFrame(df.set_index('Admit Date').reindex(idx, fill_value = 0).rename_axis('Admit Date').reset_index())

In [ ]:
df

In [ ]:
df1

In [ ]:
df_hosp = pd.concat([df, df1], ignore_index = True).sort_values(by='Admit Date').reset_index(drop = True)

In [ ]:
df_hosp

In [ ]:
df_month = df_hosp[['Admit Date', 'Daily counts']].groupby(df_hosp['Admit Date'].dt.to_period("M")).sum()


In [ ]:
df_month

In [ ]:
df_month['date'] = df_month.index

In [ ]:
#df_month.to_csv('H:/asthma/data/hosp_chk1.csv', index = False)

In [ ]:
print("Mean of the sample is % s " %(statistics.mean(df_hosp['Daily counts']))) 

In [ ]:
print("var of the sample is % s " %(statistics.variance(df_hosp['Daily counts']))) 

In [ ]:
print("Stdev of the sample is % s " %(statistics.stdev(df_hosp['Daily counts']))) 

In [ ]:
#sns.distplot(df_hosp['Daily counts'], hist=True, kde=True, 
          #   bins=int(180/5), color = 'darkblue', 
           #  hist_kws={'edgecolor':'black'},
            # kde_kws={'linewidth': 4})

In [ ]:
sns.displot(df_hosp, x="Daily counts" ,kind= "kde")

In [ ]:
sns.boxplot(df_hosp['Daily counts'])

## Removing outliers

In [ ]:
q25, q75 = percentile(df_hosp['Daily counts'], 25), percentile(df_hosp['Daily counts'], 75)
iqr = q75 - q25
print('Percentiles: 25th=%.3f, 75th=%.3f, IQR=%.3f' % (q25, q75, iqr))
# calculate the outlier cutoff
cut_off = iqr * 2
lower, upper = q25 - cut_off, q75 + cut_off
# identify outliers
outliers = [x for x in df_hosp['Daily counts'] if x < lower or x > upper]
print('Identified outliers: %d' % len(outliers))
# remove outliers
outliers_removed = [x for x in df_hosp['Daily counts'] if x >= lower and x <= upper]
print('Non-outlier observations: %d' % len(outliers_removed))

In [ ]:
upper

In [ ]:
#df_hosp[df_hosp['Daily counts'] > upper]
#df_hosp[df_hosp['Daily counts'] < lower]

In [ ]:
#new_df = df_hosp[df_hosp['Daily counts'] <= upper]

In [ ]:
#sns.boxplot(new_df['Daily counts'])

In [ ]:
new_df_cap = df_hosp.copy()
new_df_cap['Daily counts'] = np.where(
    new_df_cap['Daily counts'] > upper,
    upper,
    np.where(
        new_df_cap['Daily counts'] < lower,
        lower,
        new_df_cap['Daily counts']
    )
)

In [ ]:
new_df_cap

In [ ]:
#new_df_cap.to_csv('H:/asthma/data/hosp_chk2.csv', index = False)

In [ ]:
sns.boxplot(new_df_cap['Daily counts'])

In [ ]:
new_df_cap['scaled data'] = stats.zscore(new_df_cap['Daily counts'])

In [ ]:
new_df_cap

In [ ]:
per = np.percentile(new_df_cap ['scaled data'], 70)

In [ ]:
per

## Dividing into classes

In [ ]:
df_hosp_new = pd.DataFrame(new_df_cap.assign(status=pd.cut(new_df_cap['scaled data'], 
                                bins = [-4, 0.6260291810456466, 4],
                             # bins=[-3, 0.3381293922380649 , 0.9290142083036281, 3], 
                              #  bins = [-1, 6, 8, 55],               
                              # labels=['Low','Medium', 'High']
                                labels = ['Low', 'High']                            
                                                          
                                                          ))
                      )

In [ ]:
df_hosp_new

In [ ]:
df_hosp_new["status"].replace({"High": 1, "Low": 0}, inplace=True)

In [ ]:
df_hosp_new

In [ ]:
df_hosp

In [ ]:
#df_hosp_new.loc[df_hosp_new['status'] == 0]

In [ ]:
chk = df_hosp

In [ ]:
chk.set_index('Admit Date', inplace = True)

In [ ]:
chk


In [ ]:
a = chk[['Daily counts']].diff()

In [ ]:
a

In [ ]:
df_hosp

In [ ]:
df_month

In [ ]:
chk[['Daily counts']].plot(figsize = (20, 10), linewidth = 5, fontsize = 20)
plt.xlabel('Admit Date', fontsize= 20);

In [ ]:
chk[['Daily counts']].rolling(15).mean().plot(figsize = (20, 10), linewidth = 5, fontsize = 20)
plt.xlabel('Admit Date', fontsize= 20);

In [ ]:
#df_hosp_new.to_csv('H:/asthma/data/hosp_withoutcbu_auck.csv', index = False)
df_hosp_new.to_csv((os.path.join(dataset_dir, 'hosp_withoutcbu_auck.csv')), index = False)

In [ ]:
#df_y1 = pd.DataFrame(df_hosp.loc[(df_hosp['Admit Date'] >= '2018-01-01') & (df_hosp['Admit Date'] <= '2019-01-01')]).sort_values(by = "Admit Date").reset_index(drop=True)
#df_y2 = pd.DataFrame(df_hosp.loc[(df_hosp['Admit Date'] >= '2019-01-01') & (df_hosp['Admit Date'] <= '2020-01-01')]).sort_values(by = "Admit Date").reset_index(drop=True)
#df_y3 = pd.DataFrame(df_hosp.loc[(df_hosp['Admit Date'] >= '2020-01-01') & (df_hosp['Admit Date'] <= '2021-01-01')]).sort_values(by = "Admit Date").reset_index(drop=True)


In [ ]:
#df_y2

In [ ]:
#fig, axs = plt.subplots(3, sharex=False, sharey=False, figsize=(10,10))
#fig.suptitle('Daily counts of patients')
#axs[0].plot(df_y1['Admit Date'], df_y1['Daily counts'])
#axs[0].set_title('Daily counts of patients')
#axs[1].plot(df_y2['Admit Date'], df_y2['Daily counts'])
#axs[2].plot(df_y3['Admit Date'], df_y3['Daily counts'])
#plt.xlabel("Time Period")
#plt.ylabel("Counts")

In [ ]:
#plt.figure(3)
#plt.subplot(311)
#plt.plot(df_y1['Admit Date'], df_y1['Daily counts'])
#plt.ylabel('Daily counts')
#plt.subplot(312)
#plt.plot(df_y2['Admit Date'], df_y2['Daily counts'])
#plt.ylabel('Daily counts')
#plt.subplot(213)
#plt.plot(df_y3['Admit Date'], df_y3['Daily counts'])
#plt.ylabel('Daily counts')
#plt.show()

In [ ]:
df_plot= pd.DataFrame(df_hosp_new[['Admit Date','Daily counts']])

In [ ]:
df_plot

In [ ]:
df_plot = df_plot.rename(columns={'Admit Date': 'Date', 'Daily counts': 'counts'})

In [ ]:
df_plot

In [ ]:
#mask = (df_plot['Date'] >= '2020-01-01') & (df_plot['Date'] <= '2021-01-01')
#df_plot = df_plot.loc[mask]
#set Date as index
#commenting
#df_plot.set_index('Date', inplace=True)

In [ ]:
df_plot

In [ ]:
df_plot.info()

In [ ]:
plt.figure(figsize=(20, 10), dpi=150)
plt.plot(df_plot.Date, df_plot.counts, color='tab:blue')
plt.gca().set(title='Daily counts of asthma-related ED visits/hospitalizations from 2018 to 2021', 
xlabel='Date', ylabel='Number of patients')
plt.show()

In [ ]:
import itertools

p = d = q = range(0, 3)
#generate patterns from p,q,r
pdq = list(itertools.product(p, d, q))


In [ ]:
pdq

In [ ]:
import warnings
import statsmodels
from statsmodels.tsa.arima.model import ARIMA
warnings.filterwarnings("ignore")
for param in pdq:
        mod = ARIMA(
            df_plot['counts'],
            order=param,
            enforce_stationarity=False,
            enforce_invertibility=False)
        results = mod.fit()
        print('ARIMA{} - AIC:{}'.format(param, results.aic))

In [ ]:
df_plot['mean'] = df_plot['counts'].rolling(window=7,  min_periods=1).mean()
df_plot['mean'].plot(figsize=(20, 10), linewidth=2, y='Number of patients',
color='tab:blue', title='Daily counts of asthma-related ED visits/hospitalizations from 2018 to 2021')

In [ ]:
df_plot

In [ ]:
z = df_plot[["Date","counts", "mean"]]

In [ ]:
z.rename(columns = {'mean':'Mean_value'}, inplace = True)

In [ ]:
z

In [ ]:
actual_vals = z.counts.values


In [ ]:
train, test = actual_vals[0:548], actual_vals[548:]

In [ ]:
df_month

## Monthly analyzing

In [ ]:
df_month.drop(df_month.tail(1).index,inplace=True)

In [ ]:
w = df_month
actual_vals = w['Daily counts'].values.astype('float32')
train, test = actual_vals[0:18], actual_vals[18:]

In [ ]:
test

In [ ]:
my_order = (0, 1, 0) #(2,2,0)
my_seasonal_order = (0, 1, 1, 12)

In [ ]:
history = [x for x in train]
predictions = list()

In [ ]:
import statsmodels.api as sm
from statsmodels.tsa.arima_model import ARIMA
for t in range(len(test)):
     model = ARIMA(
                history,
                order=my_order)
     model_fit = model.fit()
     output = model_fit.forecast()
     #predict_log.append(output[0]) 
     #yhat = 10**output[0]
     predictions.append(int(output[0][0]))
     obs = test[t]
     history.append(obs)

In [ ]:
output[0][0]

In [ ]:
#import statsmodels.api as sm
#from statsmodels.tsa.arima_model import ARIMA
#for t in range(len(test)):
    # model = sm.tsa.SARIMAX(
     #           history,
      #          order=my_order, seasonal_order= my_seasonal_order,
       #         enforce_stationarity=False,
        #        enforce_invertibility=False)
   #  model_fit = model.fit()
   #  output = model_fit.forecast()
     #predict_log.append(output[0]) 
     #yhat = 10**output[0]
    # predictions.append(output)
    # obs = test[t]
     #history.append(obs)


In [ ]:
predictions

In [ ]:
predicted_df = pd.DataFrame()
predicted_df["eventDate"] = w.index[18:]
predicted_df["actual"] = test
predicted_df["Predicted"] = predictions

In [ ]:
predicted_df

In [ ]:

df_month['date'] = (df_month['date']).apply(lambda d: pd.to_datetime(str(d)))

In [ ]:
df_month

In [ ]:
predicted_df

In [ ]:
df_month[['Daily counts']].plot(figsize = (20,8), linewidth = 5, fontsize = 15)
plt.xlabel('Admit Date', fontsize= 15);

In [ ]:
predicted_df['date'] = (predicted_df['eventDate']).apply(lambda d: pd.to_datetime(str(d)))

In [ ]:
predicted_df.info()

In [ ]:
plt.figure(figsize=(12,5), dpi=100)
#plt.plot(df_month['date'], df_month['Daily counts'], label='Actual')
#plt.plot(test, label='actual')

plt.plot(predicted_df['date'],predicted_df["actual"], label='Actual')
plt.plot(predicted_df['date'],predicted_df["Predicted"], label='Predicted')
#plt.title('Forecast vs Actuals')
plt.legend(loc='upper left', fontsize=8)
plt.show()

In [ ]:
actual_df = pd.DataFrame()
actual_df["eventDate"] = w.index[0:19]
actual_df['date'] = (actual_df['eventDate']).apply(lambda d: pd.to_datetime(str(d)))
actual_df["Train_data"] = actual_vals[0:19]

In [ ]:
actual_df

In [ ]:
plt.figure(figsize=(12,5), dpi=100)
#plt.plot(df_month['date'], df_month['Daily counts'], label='Actual')
#plt.plot(test, label='actual')
plt.plot(actual_df['date'],actual_df["Train_data"], label='Train')
plt.plot(predicted_df['date'],predicted_df["actual"], label='Test')
plt.plot(predicted_df['date'],predicted_df["Predicted"], 'm--',label='Predicted')
#plt.title('Forecast vs Actuals')
plt.legend(loc='upper left', fontsize=8)
plt.show()

In [ ]:
#predicted_df = pd.DataFrame()
#predicted_df["eventDate"] = z.Date[548:]
#predicted_df["actual"] = test
#predicted_df["Predicted"] = predictions

In [ ]:
#predicted_df.to_csv('H:/asthma/data/arima.csv', index = False)

In [ ]:
#plt.figure(figsize=(12,5), dpi=100)
#plt.plot(z.Date, actual_vals, label='Actual')
#plt.plot(test, label='actual')
#plt.plot(predicted_df['eventDate'],predicted_df["actual"], label='Actual')
#plt.plot(predicted_df['eventDate'],predicted_df["actual"], label='Predicted')
#plt.title('Forecast vs Actuals')
#plt.legend(loc='upper left', fontsize=8)
#plt.show()

In [ ]:
#plt.figure(figsize=(12,5), dpi=100)
#plt.plot(z.Date, actual_vals, label='Actual')
#plt.plot(test, label='actual')
#plt.plot(predicted_df['eventDate'],predicted_df["actual"], label='Actual')
#plt.plot(predicted_df['eventDate'],predicted_df["Predicted"], label='Predicted')
#plt.title('Forecast vs Actuals')
#plt.legend(loc='upper left', fontsize=8)
#plt.show()

In [ ]:
#actual_vals = z.Mean_value.values

In [ ]:
#train, test = actual_vals[0:548], actual_vals[548:]

In [ ]:
#history = [x for x in train]
#predictions = list()

In [ ]:
#import statsmodels.api as sm
#for t in range(len(test)):
     #model = ARIMA(
                #history,
                #order=my_order)
     #model_fit = model.fit()
     #output = model_fit.forecast()
     #predict_log.append(output[0]) 
     #yhat = 10**output[0]
     ##predictions.append(output[0][0])
     #obs = test[t]
     #history.append(obs)

In [ ]:
#predictions

In [ ]:
#predicted_df = pd.DataFrame()
#predicted_df["eventDate"] = z.Date[548:]
#predicted_df["actual"] = test
#predicted_df["Predicted"] = predictions

In [ ]:
#plt.figure(figsize=(12,5), dpi=100)
#plt.plot(z.Date, z.Mean_value, label='Actual')
#plt.plot(test, label='actual')
#plt.plot(predicted_df['eventDate'],predicted_df["Predicted"], label='Predicted')
#plt.title('Forecast vs Actuals')
#plt.legend(loc='upper left', fontsize=8)
#plt.show()

In [ ]:
#mod = ARIMA(
            #train,
            #order=(0, 1, 0),
           
#)
#results = mod.fit()

In [ ]:
#print(results.summary())

In [ ]:
#df_plot['Forecast']= results.predict(start=pd.to_datetime('2019-07-01'))
#df_plot[['counts','Forecast']].plot(figsize=(20,8))


In [ ]:
#plt.figure(figsize=(20, 8), dpi=150)
#plt.plot(df_plot.index, df_plot['counts'],  label='Actual')
#plt.plot(df_plot.index, df_plot['Forecast'], label = 'Predicted')
#plt.xlabel('Date')
#plt.ylabel('Number of patients')
#plt.title('Daily counts of asthma-related ED visits/hospitalizations from 2018 to 2021')
#plt.legend()
#plt.show()

In [ ]:
#df_plot

In [ ]:
#mod1 = ARIMA(
            #df_plot['mean'],
            #order=(0, 1, 0),
           
#)
#results1 = mod1.fit()

In [ ]:
#df_plot['Forecasted']= results1.predict(start=pd.to_datetime('2019-07-01'))
#df_plot[['mean','Forecasted']].plot(figsize=(20,8))

In [ ]:
#plt.figure(figsize=(20, 8), dpi=150)
#plt.plot(df_plot.index, df_plot['mean'],  label='Actual')
##plt.plot(df_plot.index, df_plot['Forecasted'], label = 'Predicted')
#plt.xlabel('Date')
#plt.ylabel('Number of patients')
#plt.title('Weekly counts of asthma-related ED visits/hospitalizations from 2018 to 2021')
#plt.legend()
#plt.show()